In [1]:
import torch
import torch.nn as nn
from torchvision.models import vgg16, VGG16_Weights
import td_load_data
import td_run_model2
import os
import shutil
from torchvision import transforms
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_classes = 3  # Adjust this to match the number of classes (e.g., high, medium, low)
num_epochs = 23
batch_size = 16
learning_rate = 0.005

In [4]:
class VGG16MultiClassClassifier(nn.Module):
    def __init__(self, num_classes):
        super(VGG16MultiClassClassifier, self).__init__()
        self.base_model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        
        in_features = self.base_model.classifier[6].in_features
        
        self.base_model.classifier[6] = nn.Sequential(
            nn.Linear(in_features, num_classes)
        )

    def forward(self, x):
        x = self.base_model(x)
        return x

In [ ]:
if __name__ == "__main__":
    # Hyperparameters

    # Model initialization
    model = VGG16MultiClassClassifier(num_classes=num_classes).to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # Load data
    train_loader, validation_loader, test_loader, classes = td_load_data.create_data(batch_size=batch_size)
    
    # Test the model's forward pass with a sample input
    sample_input = torch.randn(1, 3, 224, 224).to(device)
    print("Output shape:", model(sample_input).shape)  # This should now be [1, num_classes]
    
    # Train and validate the model
    td_run_model2.train(num_epochs, device, model, criterion, optimizer, train_loader, validation_loader, num_classes)
    
    
    # Test the model
    td_run_model2.test(device, model, test_loader, num_classes)


In [5]:
# Define the model architecture with the same number of classes used during training
model = VGG16MultiClassClassifier(num_classes=num_classes).to(device)

# Load the state dict
# state_dict = torch.load('weights/43.pth')
# state_dict = torch.load('weights/42.pth')
state_dict = torch.load('weights/43.pth', map_location=torch.device('cpu'))     # Running locally

# Load the state dict into the model
model.load_state_dict(state_dict)

model.eval()  # Set the model to evaluation mode

# Load the test data using the function in td_load_data.py
_, _, test_loader, classes = td_load_data.create_data(batch_size=batch_size)

# Evaluate the model on the test set
td_run_model2.test_model_on_test_data(device, model, test_loader)

C:\Users\Abiya\AppData\Local\Temp\ipykernel_18908\571911711.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('weights/43.pth', map_location=torch.

UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='C:/dDrive/CS480-fyp/traffic_density/demo\\high\\cars-city-traffic-daylight_23-2149092084.jpeg'>

In [ ]:
#takes an avg of 3m 40s to run

# Define the paths
input_folder = "C:/dDrive/CS480-fyp/traffic_density/8702"  # Path to the folder with unsorted photos
output_folder = "sorted_photos/"   # Base path to save sorted photos
high_folder = os.path.join(output_folder, "high")
medium_folder = os.path.join(output_folder, "medium")
low_folder = os.path.join(output_folder, "low")

# Create output directories if they don't exist
os.makedirs(high_folder, exist_ok=True)
os.makedirs(medium_folder, exist_ok=True)
os.makedirs(low_folder, exist_ok=True)

model = VGG16MultiClassClassifier(num_classes=num_classes).to(device)  # Change this based on your specific model

model.load_state_dict(torch.load('weights/43.pth', map_location=torch.device('cpu')))  # Path to your weights

model.to(device)
model.eval()  # Set the model to evaluation mode

# Define the transformation (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to predict class for a single image
def predict_image(image_path, model, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()  # Return the predicted class index (0, 1, 2 for high, medium, low)

# Loop through each image in the input folder
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    if os.path.isfile(img_path):
        predicted_class = predict_image(img_path, model, device)
        
        # Move image to the corresponding class folder
        if predicted_class == 0:  # Assuming 0 is high, 1 is medium, 2 is low
            shutil.move(img_path, os.path.join(high_folder, img_name))
        elif predicted_class == 1:
            shutil.move(img_path, os.path.join(low_folder, img_name))
        elif predicted_class == 2:
            shutil.move(img_path, os.path.join(medium_folder, img_name))
        

print("Photos have been successfully sorted into high, medium, and low folders.")

###### 